* Dataset
* Build model
* Define loss function and optimizer
* Define trainer (model produces prediction -> compute the loss (label - pred) -> backward pass)
* Define test (on validation) -> training/validation
* Run trainer and test

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


In [ ]:
# get the dataset
train_ds = MNIST(root='data', train=True, download=True, transform=ToTensor())
valid_ds = MNIST(root='data', train=False, download=True, transform=ToTensor())

In [ ]:
# Examine the data

image, label = train_ds[0]
print(label)
plt.imshow(image.float().reshape(28, 28), cmap='gray')

In [ ]:
# Create a data loader
# Makes it easier to iterate over batches
# GPU accelration -> num_workers = 4, pin_memory (pinned memory -> faster RAM to gpu transfer)
bs = 64
train_dl = DataLoader(train_ds, batch=bs, shuffle=True)
valid_dl = DataLoader(valid_bs, batch=bs, shuffle=True)

In [ ]:
from torch import nn

# build a model
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__
        self.lin = nn.Linear(784, 10)
    
    def forward(self, xb):
        xb.flatten(1, -1) # (bs, 1, 28, 28) -> (bs, 784)
        return self.lin(xb)

model = MNISTModel()
print(model)


        